# SMAP-USDA monthly averages

Monthly climatologies/stats of 
soil moisture quantities and 
their derivatives (porosity and available porosity)

James McCreight 


In [1]:
from bokeh.io import output_notebook 
from bokeh.plotting import figure, output_file, show 
from bokeh.palettes import brewer, colorblind
from IPython.display import Image
output_notebook()

import ee

In [2]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g5-6Adt57xPRykgTyvhENzTVDzZjc3SXoeMGoZdyv2WpjebsQ3j7ME



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
smap_usda = (ee
  .ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture'))

In [5]:
bands = ee.List(['smp', 'ssm', 'susm'])
months = ee.List.sequence(1, 12);

In [6]:
# monthly averages for each band
def band_mon_avg(bb):
  def mon_avg(mm):
    return (smap_usda
           .filter(ee.Filter.calendarRange(mm, mm, 'month'))
           .select([bb])
           .mean()
           .set('month', mm).set('band', bb))
  return ee.ImageCollection.fromImages(months.map(mon_avg).flatten())

In [7]:
monthly_means = bands.map(band_mon_avg)

In [8]:
# print(monthly_means)

In [9]:
monthly_means.getInfo()

[{'type': 'ImageCollection', 'bands': []},
 {'type': 'ImageCollection', 'bands': []},
 {'type': 'ImageCollection', 'bands': []}]

In [10]:
# separate the variables
smp = ee.ImageCollection(monthly_means.get(0))
ssm = ee.ImageCollection(monthly_means.get(1))
susm = ee.ImageCollection(monthly_means.get(2))

In [11]:
# porosity in mm (millimeters)
def mon_porosity_calc(mm):
  porosity = (ssm
    .filter(ee.Filter.eq('month', mm)).first()
    .add(susm.filter(ee.Filter.eq('month', mm)).first())
    .divide(smp.filter(ee.Filter.eq('month', mm)).first())
    .set('month', mm).set('band', 'porosity_mm'))
  return ee.Image(porosity)

porosity = ee.ImageCollection(months.map(mon_porosity_calc))
# porosity.getInfo()

In [12]:
# available/additional/unused porosity
def mon_avail_porosity_calc(mm):
  avail_porosity = (porosity
    .filter(ee.Filter.eq('month', mm)).first()
    .subtract(
      ssm.filter(ee.Filter.eq('month', mm)).first()
      .add(susm.filter(ee.Filter.eq('month', mm)).first()))
      .set('month', mm).set('band', 'avail_porosity_mm'))
  return ee.Image(avail_porosity)

avail_porosity = ee.ImageCollection(months.map(mon_avail_porosity_calc))
# avail_porosity.getInfo()

In [13]:
# Export to images, build a collection
smap_collection = ee.ImageCollection(
  ee.List([
    ssm.toList(12),
    susm.toList(12),
    smp.toList(12),
    porosity.toList(12),
    avail_porosity.toList(12)]).flatten())

# smap_collection.getInfo()

In [14]:
# smap_asset = ee.data.createAsset({'type': 'ImageCollection'}, 'users/jamesmcc/smap_usda_climatology')
smap_asset = {'id': 'users/jamesmcc/smap_usda_climatology'}

In [15]:
smap_asset

{'id': 'users/jamesmcc/smap_usda_climatology'}

In [28]:
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# bands = ['smp'] #, 'ssm', 'susm']
bands = ['porosity_mm', 'avail_porosity_mm']
bbox = ee.Geometry.BBox(-180.0, -90, 180, 90)
for bb in bands:
    for mm0 in range(12):
        mm = mm0 + 1
        description = 'SMAP-USDA_mon_clim_' + str(bb) + '_' + str(mm).zfill(2) + '_' + month_names[mm0]
        xx = ee.batch.Export.image.toAsset(
            smap_collection
            .filter(ee.Filter.eq('month', mm))
            .filter(ee.Filter.eq('band', bb)).first(),
            description=description, 
            assetId=smap_asset['id'] + '/' + description,
            region=bbox).start()

In [20]:
# -------------------------

In [17]:
palette_name = 'BrBG'
palette_len = 11
palette = brewer[palette_name][palette_len][::-1]  # reverse it

In [30]:
legend = figure(title=palette_name, plot_width=400, plot_height=75)  
# demonstrating the BrBG palette 
legend.vbar(x = range(palette_len),  
           top = [1] * palette_len, 
           bottom = [0] * palette_len, 
           width = 1, 
           color = palette) 
show(legend)

In [27]:
# Display thumbnails

bb = 'avail_porosity_mm'

# How to get the min/max over the full climatology
vis_params = {
    'min': 0, 'max': 1, 'dimensions': 512,
    'palette': palette}

for mm0 in range(1):
    mm = mm0 + 1
    img = (
        smap_collection
        .filter(ee.Filter.eq('month', mm))
        .filter(ee.Filter.eq('band', bb)).first())
    display(Image(url = img.getThumbURL(vis_params)))